https://nces.ed.gov/ccd/elsi/tableGenerator.aspx

https://www.usinflationcalculator.com/inflation/current-inflation-rates/
https://data.bls.gov/pdq/SurveyOutputServlet

In [1]:
from collections import namedtuple

import polars as pl
import plotly.express as px
import plotly.graph_objects as go
import statsmodels.regression

In [ ]:
df = pl.read_csv(
    "./data/nces/geo/ELSI/state/ELSI_csv_export_6385899184786235507585.csv",
    skip_rows=6,
    n_rows=51
).drop("State Name [State] 2018-19").with_columns(
    pl.col("State Name").str.to_titlecase()
)
df = df.unpivot(
    on=df.columns[1:],
    index="State Name",
    variable_name="VARIABLE",
    value_name="VALUE"
).with_columns(
    SCHOOL_YEAR=pl.col("VARIABLE").str.extract(r"(\d{4}\-\d{2})$"),
    VARIABLE=pl.col("VARIABLE").str.replace(r" \[(State( Finance)?)\] (\d{4}\-\d{2})$", ""),
    VALUE=pl.col("VALUE").replace(chr(8211), None).str.to_integer(),
).pivot(
    on="VARIABLE",
    index=["State Name", "SCHOOL_YEAR"],
    values="VALUE",
    aggregate_function="first"
).with_columns(
    pl.sum_horizontal("Kindergarten Students", "Prekindergarten Students", "Grades 1-8 Students", "Grades 9-12 Students").alias("Total Students"),
    pl.sum_horizontal("Grades 1-8 Students", "Grades 9-12 Students").alias("Grades 1-12 Students")
).sort("State Name", "SCHOOL_YEAR").with_columns(
    pl.col("SCHOOL_YEAR").str.extract(r"^(\d{4})").str.to_integer().alias("SCHOOL_YEAR_START"),
    pl.col("SCHOOL_YEAR").str.extract(r"\-(\d{2})$").str.to_integer().add(2000).alias("SCHOOL_YEAR_END")
)
df

In [ ]:
inflation = pl.read_excel(
    "./data/bls/inflation/SeriesReport-20240811173715_e1eda9.xlsx",
    read_options=dict(header_row=11),
)
inflation = inflation.with_columns(
    pl.col("HALF1").shift(-1).alias("HALF1 (Next Year)")
).filter(pl.col("Year") < 2024).sort("Year", descending=True).with_columns(
    (1 + pl.col("HALF1 (Next Year)") / 100).cum_prod().alias("Current to 2024 Dollar Conversion Rate")
).sort("Year")
inflation

In [ ]:
student_investment = df.join(
    inflation.select("Year", "Current to 2024 Dollar Conversion Rate"),
    left_on="SCHOOL_YEAR_END",
    right_on="Year",
).with_columns(
    (
        pl.col("Total Expenditures (TE11+E4D+E7A1) per Pupil (MEMBR)")
        * pl.col("Current to 2024 Dollar Conversion Rate")
    ).alias("Total Expenditures (TE11+E4D+E7A1) per Pupil (MEMBR) (2024 Dollars)")
).with_columns(
    pl.col("Total Expenditures (TE11+E4D+E7A1) per Pupil (MEMBR) (2024 Dollars)")
    .rolling_sum(4)
    .over("State Name")
    .alias("Total Expenditures (TE11+E4D+E7A1) per Graduating Pupil (MEMBR) (4-Year Rolling Sum) (2024 Dollars)"),
).select(
    [
        "State Name",
        "SCHOOL_YEAR",
        "Total Expenditures (TE11+E4D+E7A1) per Graduating Pupil (MEMBR) (4-Year Rolling Sum) (2024 Dollars)"
    ]
).filter(
    pl.col("SCHOOL_YEAR").is_in(["2014-15", "2018-19"])
).pivot(
    on="SCHOOL_YEAR",
    index="State Name",
    values="Total Expenditures (TE11+E4D+E7A1) per Graduating Pupil (MEMBR) (4-Year Rolling Sum) (2024 Dollars)",
).with_columns(
    (pl.col("2018-19") - pl.col("2014-15")).alias("Investment Change")
).sort("Investment Change")
student_investment

In [ ]:
math_performance = pl.concat(
    [
        pl.read_csv(
            "data/nces/geo/nations-report-card/2019/state_grade-8_math_SPCsv202408081556.csv",
            columns=["Jurisdiction", "MN", "AB", "AP"],
        ).with_columns(
            pl.lit(2015).alias("Graduating Year")
        ),
        pl.read_csv(
            "data/nces/geo/nations-report-card/2015/state_grade-8_math_SPCsv202408081407.csv",
            columns=["Jurisdiction", "MN", "AB", "AP"],
        ).with_columns(
            pl.lit(2019).alias("Graduating Year"),
            pl.col("AP").replace("#", None).str.to_integer()
        ),
    ]
).rename(
    {
        "Jurisdiction": "State Name",
        "MN": "Mean Grade 8 Math Score",
        "AB": "At or Above Basic (%)",
        "AP": "At or Above Proficient (%)",
    }
).pivot(
    on="Graduating Year",
    index="State Name",
    values=["Mean Grade 8 Math Score", "At or Above Basic (%)", "At or Above Proficient (%)"],
).with_columns(
    (pl.col("Mean Grade 8 Math Score_2019") - pl.col("Mean Grade 8 Math Score_2015")).alias("Mean Grade 8 Math Score Change"),
    (pl.col("At or Above Basic (%)_2019") - pl.col("At or Above Basic (%)_2015")).alias("At or Above Basic (%) Change"),
    (pl.col("At or Above Proficient (%)_2019") - pl.col("At or Above Proficient (%)_2015")).alias("At or Above Proficient (%) Change"),
).select(
    "State Name",
    "Mean Grade 8 Math Score Change",
    "At or Above Basic (%) Change",
    "At or Above Proficient (%) Change",
).sort("Mean Grade 8 Math Score Change")

math_performance

In [6]:
Result = namedtuple("CorrelationResult", ["result", "r_squared", "p_value", "coef"])

def correlate(df: pl.DataFrame, x: str, y: str, show_summary: bool = True) -> Result:
    """Return the OLS Adjusted R-squared and p-value of the model."""
    result = statsmodels.regression.linear_model.OLS(
        endog=df[y].to_list(),
        exog=df[x].to_list(),
    ).fit()
    if show_summary:
        display(result.summary())
    return Result(
        result=result,
        r_squared=float(result.rsquared_adj),
        p_value=float(result.pvalues[0]),
        coef=float(result.params[0]),
    )

In [7]:
# correlate(df, "Investment Change", "Mean Grade 8 Math Score Change")

In [ ]:
def generate_state_investment_and_math_performance_scatter_plot(column: str, yaxis_title: str, exclude_alaska: bool = True) -> go.Figure:
    _df = student_investment.join(
        math_performance,
        on="State Name"
    ).join(
        # Adding the number of students in the 8th grade cohort
        df.filter(pl.col("SCHOOL_YEAR").is_in(["2018-19"])).select("State Name", "Grade 8 Students"),
        on="State Name",
        suffix="_students",
    ).with_columns(
        # Only displaying state names whose values stand out
        # Of the top states in investment/score changes, choose the top ranks among them
        pl.when(
            pl.max_horizontal(
                (pl.col(column) - pl.col(column).mean()).abs().rank(method="max"),
                (pl.col("Investment Change") - pl.col("Investment Change").mean()).abs().rank(method="max")
            ).rank(method="max").is_between(35, 50)
        )
        .then(pl.col("State Name"))
        .otherwise(None)
        .alias("Display Name")
    )
    result = correlate(
        _df.filter(pl.col("State Name") != ("Alaska" if exclude_alaska else None)),
        "Investment Change",
        column,
    )
    return px.scatter(
        _df,
        x="Investment Change",
        y=column,
        size="Grade 8 Students",
        text="Display Name",
        title=(
            "Increasing Student Investment is a bad approach for raising Math Performance"
            "<br>"
            "<sup>"
            "Comparison of 2015 & 2019's 8<sup>th</sup> grade graduating classes across 50 U.S. States"
            "</sup>"
        ),
        labels={
            "2014-15": "Per Pupil Investment (2014-15)",
            column: yaxis_title,
        },
        hover_data=["State Name"],
        hover_name="State Name",
        template="plotly_dark",
        width=1280,
        height=720,
    ).update_layout(
        # coloraxis_cmid=0,
        xaxis_title_font_size=10,
        xaxis_title="Change in Student Investment<br><sup>(2024 Dollars)</sup>",
        coloraxis_colorbar_orientation="h",
        plot_bgcolor="#171717",
        paper_bgcolor="#171717",
        margin=dict(t=50, b=135),
    ).update_traces(
        dict(
            textposition="top center",
            textfont=dict(color="grey"),
            marker=dict(color="#e69138"),
        ),
    ).add_annotation(
        # Right, watermark and source
        x=1.06,
        y=-0.230,
        xref="paper",
        yref="paper",
        text="<br>".join(
            (
                "Chart by Dominic Tarro | 𝕏 @dominictarro",
                "Sources: NAEP Mathematics Assessments (2015, 2019)",
                "CCD National Public Education Financial Survey (2011-2019), Series TE11+E4D+E7A1",
            )
        ),
        align="right",
        showarrow=False,
        font=dict(
            size=10,
            color="grey"
        ),
        opacity=0.7
    ).add_annotation(
        x=-0.058,
        y=-0.230,
        xref="paper",
        yref="paper",
        text="<br>".join(
            (
                "Notes: The NAEP Mathematics scale scores between 0 to 500.",
                "\"Student Investment\" is the sum of real, per-pupil spending from 5th through 8th grade.",
                "Dollar adjustment was made using each school year's ending year.",
                "OLS Adj. R-squared: {:.03f}; P|t|>{:.03f}; {:.2f} {} improvement per additional $1,000 in Student Investment{}".format(
                    result.r_squared,
                    result.p_value,
                    result.coef * 1000,
                    "points" if "mean" in column.lower() else "% points",
                    " (excl. Alaska)" if exclude_alaska else ""
                ),
            )
        ),
        align="left",
        showarrow=False,
        font=dict(
            size=10,
            color="grey"
        ),
        opacity=0.7
    ).add_hline(
        y=math_performance["Mean Grade 8 Math Score Change"].mean(),
        annotation=dict(
            text=f"Mean ({math_performance['Mean Grade 8 Math Score Change'].mean():.2f} points)",
            showarrow=False,
            font=dict(color="#76a5af"),
            yshift=5,
        ),
        annotation_position="top left",
        line=dict(
            color="#76a5af",
            width=1,
            dash="dash"
        ),
    ).add_vline(
        x=student_investment["Investment Change"].mean(),
        annotation=dict(
            text=f"Mean (${student_investment['Investment Change'].mean():,.0f})",
            showarrow=False,
            font=dict(color="#93c47d"),
            yshift=5,
            xshift=5,
        ),
        annotation_position="bottom right",
        line=dict(
            color="#93c47d",
            width=1,
            dash="dash"
        ),
    )

fig_scatter_math_avg = generate_state_investment_and_math_performance_scatter_plot("Mean Grade 8 Math Score Change", "Change in Average Math Score")
fig_scatter_math_avg.write_image(
    "charts/spending-school-roi-scatter-math-mean.svg",
)
fig_scatter_math_avg.show()

In [ ]:

def generate_state_investment_and_math_performance_bar_chart(
    column: str, legend_title: str, description: str, change_measure: str
) -> go.Figure:
    df = student_investment.join(
        math_performance,
        on="State Name"
    ).sort("Investment Change")
    result = correlate(df, "Investment Change", column)
    return px.bar(
        df,
        x="State Name",
        y="Investment Change",
        color=column,
        title=f"US Primary Education Return on Investment<br><sup><i>{description}</i></sup>",
        template="plotly_dark",
        color_continuous_scale=px.colors.sequential.RdBu,
        labels={
            "State Name": "State",
            "Investment Change": "4-Year Expenditure Change (2024 Dollars)",
            column: f"Change in NAEP {legend_title}",
        },
        width=1000,
        height=600,
    ).update_layout(
        xaxis=dict(showticklabels=False, title=None),
        yaxis_title="Per Pupil, 4-Year Expenditure Change Between 2015 and 2019<br>(2024 Dollars)",
        yaxis_title_font_size=10,
        coloraxis_colorbar_title=f"Change in <i>NAEP {legend_title}</i>",
        coloraxis_colorbar_orientation="h",
        plot_bgcolor="#171717",
        paper_bgcolor="#171717",
    ).update_coloraxes(
        cmid=0,
    ).add_annotation(
        x=1.07,
        y=-0.15,
        xref="paper",
        yref="paper",
        text="<br>".join(
            (
                "Chart by Dominic Tarro | 𝕏 @dominictarro",
                "Sources: NAEP Mathematics Assessments (2015, 2019)",
                "CCD National Public Education Financial Survey (2010-2019)"
            )
        ),
        align="right",
        showarrow=False,
        font=dict(
            size=10,
            color="grey"
        ),
        opacity=0.7
    ).add_annotation(
        x=-0.08,
        y=-0.15,
        xref="paper",
        yref="paper",
        text="<br>".join(
            (
                "Note: The NAEP Mathematics scale ranges from 0 to 500.",
                "<i>Per Pupil, 4-Year Expenditure</i> is an aggregate from 5th-8th grade. Dollar adjustment was made using <br>each school year's ending year.",
                f"OLS Adj. R-squared: {result.r_squared:.03f}; P|t|>{result.p_value:.03f}; {change_measure.title()} change per $1000 change in per student spending={result.coef * 1000:.2f}"
            )
        ),
        align="left",
        showarrow=False,
        font=dict(
            size=10,
            color="grey"
        ),
        opacity=0.7
    )

_fig1 = generate_state_investment_and_math_performance_bar_chart(
    "Mean Grade 8 Math Score Change",
    "Average Math Score",
    "States spending more per student from 5th through 8th grade had a negligible relationship with average 8th grade math performance.",
    "score",
)
_fig1.write_image("charts/spending-school-roi-math-mean.png")
_fig1.show()
_fig2 = generate_state_investment_and_math_performance_bar_chart(
    "At or Above Basic (%) Change",
    "At or Above Basic</i><br><sup>(% Points)</sup>",
    "States spending more per student from 5th through 8th grade had a small relationship with basic 8th grade math performance.",
    "score",
)
_fig2.write_image("charts/spending-school-roi-math-basic.png")
_fig2.show()
_fig3 = generate_state_investment_and_math_performance_bar_chart(
    "At or Above Proficient (%) Change",
    "At or Above Proficient</i><br><sup>(% Points)</sup>",
    "States spending more per student from 5th through 8th grade had a negligible relationship with 8th grade math profieciency.",
    "score",
)
_fig3.write_image("charts/spending-school-roi-math-proficient.png")
_fig3.show()

In [ ]:
reading_performance = pl.concat(
    [
        pl.read_csv(
            "data/nces/geo/nations-report-card/2019/state_grade-8_reading_SPCsv202408081536.csv",
            columns=["Jurisdiction", "MN", "AB", "AP"],
        ).with_columns(
            pl.lit(2015).alias("Graduating Year")
        ),
        pl.read_csv(
            "data/nces/geo/nations-report-card/2015/state_grade-8_reading_SPCsv202408081449.csv",
            columns=["Jurisdiction", "MN", "AB", "AP"],
        ).with_columns(
            pl.lit(2019).alias("Graduating Year"),
        ),
    ]
).with_columns(
            pl.col("MN").replace(chr(8212), None).str.to_integer(),
            pl.col("AB").replace(chr(8212), None).str.to_integer(),
            pl.col("AP").replace(chr(8212), None).str.to_integer(),

).rename(
    {
        "Jurisdiction": "State Name",
        "MN": "Mean Grade 8 Reading Score",
        "AB": "At or Above Basic (%)",
        "AP": "At or Above Proficient (%)",
    }
).pivot(
    on="Graduating Year",
    index="State Name",
    values=["Mean Grade 8 Reading Score", "At or Above Basic (%)", "At or Above Proficient (%)"],
).with_columns(
    (pl.col("Mean Grade 8 Reading Score_2019") - pl.col("Mean Grade 8 Reading Score_2015")).alias("Mean Grade 8 Reading Score Change"),
    (pl.col("At or Above Basic (%)_2019") - pl.col("At or Above Basic (%)_2015")).alias("At or Above Basic (%) Change"),
    (pl.col("At or Above Proficient (%)_2019") - pl.col("At or Above Proficient (%)_2015")).alias("At or Above Proficient (%) Change"),
).select(
    "State Name",
    "Mean Grade 8 Reading Score Change",
    "At or Above Basic (%) Change",
    "At or Above Proficient (%) Change",
).sort("Mean Grade 8 Reading Score Change")

reading_performance

In [ ]:
def generate_state_investment_and_reading_performance_bar_chart(
    column: str, legend_title: str, description: str, change_measure: str
) -> go.Figure:
    df = student_investment.join(
        reading_performance,
        on="State Name"
    ).sort("Investment Change")
    result = correlate(df, "Investment Change", column)
    return px.bar(
        df,
        x="State Name",
        y="Investment Change",
        color=column,
        title=f"US Primary Education Return on Investment<br><sup><i>{description}</i></sup>",
        template="plotly_dark",
        color_continuous_scale=px.colors.sequential.RdBu,
        labels={
            "State Name": "State",
            "Investment Change": "4-Year Expenditure Change (2024 Dollars)",
            column: f"Change in NAEP {legend_title}",
        },
        width=1000,
        height=600,
    ).update_layout(
        xaxis=dict(showticklabels=False, title=None),
        yaxis_title="Per Pupil, 4-Year Expenditure Change Between 2015 and 2019<br>(2024 Dollars)",
        yaxis_title_font_size=10,
        coloraxis_colorbar_title=f"Change in <i>NAEP {legend_title}</i>",
        coloraxis_colorbar_orientation="h",
        plot_bgcolor="#171717",
        paper_bgcolor="#171717",
    ).update_coloraxes(
        cmid=0,
    ).add_annotation(
        x=1.07,
        y=-0.15,
        xref="paper",
        yref="paper",
        text="<br>".join(
            (
                "Chart by Dominic Tarro | 𝕏 @dominictarro",
                "Sources: NAEP Reading Assessments (2015, 2019)",
                "CCD National Public Education Financial Survey (2010-2019)"
            )
        ),
        align="right",
        showarrow=False,
        font=dict(
            size=10,
            color="grey"
        ),
        opacity=0.7
    ).add_annotation(
        x=-0.08,
        y=-0.15,
        xref="paper",
        yref="paper",
        text="<br>".join(
            (
                "The NAEP Reading scale ranges from 0 to 500.",
                "<i>Per Pupil, 4-Year Expenditure</i> is an aggregate from 5th-8th grade. Dollar adjustment was made using <br>each school year's ending year.",
                f"OLS Adj. R-squared: {result.r_squared:.03f}; P|t|>{result.p_value:.03f}; {change_measure.title()} change per $1000 in per student spending={result.coef * 1000:.2f}"
            )
        ),
        align="left",
        showarrow=False,
        font=dict(
            size=10,
            color="grey"
        ),
        opacity=0.7
    )

_fig4 = generate_state_investment_and_reading_performance_bar_chart(
    "Mean Grade 8 Reading Score Change",
    "Average Reading Score",
    "States spending more per student from 5th through 8th grade had a negligible relationship with average 8th grade reading performance.",
    "% pts",
)
_fig4.write_image("./charts/spending-school-roi-reading-mean.png")
_fig4.show()
_fig5 = generate_state_investment_and_reading_performance_bar_chart(
    "At or Above Basic (%) Change",
    "At or Above Basic</i><br><sup>(% Points)</sup>",
    "States spending more per student from 5th through 8th grade had a small relationship with basic 8th grade reading performance.",
    "score",
)
_fig5.write_image("./charts/spending-school-roi-reading-basic.png")
_fig5.show()
_fig6 = generate_state_investment_and_reading_performance_bar_chart(
    "At or Above Proficient (%) Change",
    "At or Above Proficient</i><br><sup>(% Points)</sup>",
    "States spending more per student from 5th through 8th grade had a negligible relationship with 8th grade reading proficiency.",
    "% pts",
)
_fig6.write_image("./charts/spending-school-roi-reading-proficient.png")
_fig6.show()

In [ ]:
"${:,.2f}".format(
    student_investment.join(
        df.filter(pl.col("SCHOOL_YEAR") == "2018-19").select("State Name", "Grade 8 Students"),
        on="State Name"
    ).with_columns(
        (pl.col("Investment Change") * pl.col("Grade 8 Students")).alias("Total Investment Change"),
    )["Total Investment Change"].sum()
)

In [ ]:
math_performance["Mean Grade 8 Math Score Change"].mean(), reading_performance["Mean Grade 8 Reading Score Change"].mean()

In [ ]:
math_performance["At or Above Basic (%) Change"].mean(), reading_performance["At or Above Basic (%) Change"].mean()

In [ ]:
math_performance["At or Above Proficient (%) Change"].mean(), reading_performance["At or Above Proficient (%) Change"].mean()